 # Creating New data set

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as ctx
from shapely.geometry import Point, Polygon
from pyproj import CRS
from sklearn.linear_model import LinearRegression
import seaborn as sns
import statsmodels.api as sm
import folium
from folium.plugins import MarkerCluster, FastMarkerCluster, HeatMap#, heat_map_withtime, HeatMapWithTime
from folium.plugins.heat_map_withtime import HeatMapWithTime
from sklearn.neighbors import KernelDensity

In [3]:
# Load the dataset into a DataFrame
df = pd.read_csv('../data/kc_house_data.csv')

In [7]:
df.columns

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'greenbelt', 'nuisance', 'view',
       'condition', 'grade', 'heat_source', 'sewer_system', 'sqft_above',
       'sqft_basement', 'sqft_garage', 'sqft_patio', 'yr_built',
       'yr_renovated', 'address', 'lat', 'long', 'geometry'],
      dtype='object')

importing relevent shapefiles

In [4]:
# Read in shp
king_boundary = gpd.read_file('Metropolitan_King_County_Council___kccdst_area_copy/Metropolitan_King_County_Council___kccdst_area.shp')


Converting df to gdf

In [5]:
gdf = gpd.GeoDataFrame(df, 
                       geometry=gpd.points_from_xy(df['long'], df['lat']), 
                       crs= CRS.from_epsg(4326))

Spatially joinning dataframes

In [6]:
gdf_final = gpd.sjoin(gdf, king_boundary, how='inner')